# Extending the Model class to support OpenVINO models

## Introduction

In this tutorial we will see:
- How to add support for querying OpenVINO models
- How to load and query an OpenVINO model from a checkpoint file

## Imports

In [1]:
import numpy as np
from copy import deepcopy
import tensorflow as tf
import tensorflow_datasets as tfds

Follow one of the [OpenVINO installation tutorials](https://docs.openvino.ai/latest/openvino_docs_install_guides_install_runtime.html) specific to your platform to install the OpenVINO library.

In [2]:
from openvino.runtime import Core

For now we install the Privacy Meter library from the local source. A version will be pushed to pip soon.

In [3]:
import sys
!{sys.executable} -m pip install -e ../.
from privacy_meter.model import Model

## Extending the Model class

To add support for querying OpenVINO models, we shall extend the `Model` class of Privacy Meter. 

Additional arguments to set the device for inference and input shape accepted by the model are specified in the `OpenVinoModel` class:

In [4]:
class OpenVinoModel(Model):
    def __init__(self, model_obj, loss_fn,
                 input_shape=None,
                 device_name=None):
            """Constructor
            Args:
                model_obj: model object
                loss_fn: loss function
            """
            # Initializes the parent model
            super().__init__(model_obj, loss_fn)
            
            # Set initial input shape if provided
            if input_shape is not None:
                input_layer = next(iter(self.model_obj.inputs))
                self.model_obj.reshape({input_layer.any_name: input_shape})
                
            # Set device
            if device_name is None:
                self.device_name = 'CPU'
            else:
                self.device_name = device_name
                
            # Create a second loss function, per point
            self.loss_fn_no_reduction = deepcopy(loss_fn)
            self.loss_fn_no_reduction.reduction = 'none'


    def get_outputs(self, batch_samples):
        """Function to get the model output from a given input.
        Args:
            batch_samples: Model input
        Returns:
            Model output
        """
        model_obj = self.model_obj
        input_layer = next(iter(model_obj.inputs))
        
        # get current input shape
        current_input_shape = input_layer.get_partial_shape()

        # create new input shape with batch_size = len(batch_samples)
        new_input_shape = current_input_shape
        new_input_shape[0] = len(batch_samples)

        # reshape network with new input shape
        model_obj.reshape({input_layer.any_name: new_input_shape})
        
        # compile model before inference
        compiled_model_obj = ie.compile_model(model=model_obj, device_name=self.device_name)

        # get predictions
        output_layer = next(iter(compiled_model_obj.outputs))
        outputs = compiled_model_obj(inputs=[batch_samples])[output_layer]
    
        return outputs

    def get_loss(self, batch_samples, batch_labels, per_point=True):
        """Function to get the model loss on a given input and an expected output.
        Args:
            batch_samples: Model input
            batch_labels: Model expected output
            per_point: Boolean indicating if loss should be returned per point or reduced
        Returns:
            The loss value, as defined by the loss_fn attribute.
        """
        outputs = self.get_outputs(batch_samples)

        if per_point:
            return self.loss_fn_no_reduction(batch_labels, outputs).numpy()
        else:
            return self.loss_fn(batch_labels, outputs).numpy()


    def get_grad(self, batch_samples, batch_labels):
        """Function to get the gradient of the model loss with respect to the model parameters, on a given input and an
        expected output.
        Args:
            batch_samples: Model input
            batch_labels: Model expected output
        Returns:
            A list of gradients of the model loss (one item per layer) with respect to the model parameters.
        """
        pass

    def get_intermediate_outputs(self, layers, batch_samples, forward_pass=True):
        """Function to get the intermediate output of layers (a.k.a. features), on a given input.
        Args:
            layers: List of integers and/or strings, indicating which layers values should be returned
            batch_samples: Model input
            forward_pass: Boolean indicating if a new forward pass should be executed. If True, then a forward pass is
                executed on batch_samples. Else, the result is the one of the last forward pass.
        Returns:
            A list of intermediate outputs of layers.
        """
        pass

## Loading an OpenVINO model

Download and store the `.xml` and `.bin` checkpoint files of your OpenVINO model. 

For this tutorial, the `classification.xml` and `classification.bin` files from the [OpenVINO API Tutorial](https://github.com/openvinotoolkit/openvino_notebooks/tree/main/notebooks/002-openvino-api) have been downloaded and stored in `privacy_meter/docs/models/`.

We load the OpenVINO model from the checkpoint files:

In [5]:
ie = Core()
model_xml_filepath = "./models/classification.xml"
model = ie.read_model(model=model_xml_filepath)

Then we define the required arguments for wrapping the model into the `OpenVinoModel` object compatible with Privacy Meter:

In [6]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
device_name = "CPU"

Finally, we create the `OpenVinoModel` object:

In [7]:
openvino_model = OpenVinoModel(
    model_obj=model,
    loss_fn=loss_fn,
    device_name=device_name
)

## Querying an OpenVINO model

The following code tests if the OpenVINO model can be queried successfully.

### Load ImageNet A from Tensorflow Datasets

In [8]:
def resize_img(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    channels_first_image = tf.transpose(resized_image, [2, 0, 1])
    return channels_first_image, label
    
ds = tfds.load('imagenet_a', split='test', as_supervised=True)
ds = ds.map(resize_img)

In [9]:
num_samples = 500

x, y, ctr = [], [], 0
for (image, label) in tfds.as_numpy(ds):
    x.append(image)
    y.append(label)
    ctr = ctr + 1
    
    if ctr == num_samples:
        break
    
x = np.array(x)
y = tf.keras.utils.to_categorical(y, num_classes=1001)

### Get Outputs and Loss Values

In [10]:
outputs = openvino_model.get_outputs(x)

In [11]:
outputs.shape

(500, 1001)

In [12]:
loss_values = openvino_model.get_loss(x, y, per_point=True)

In [13]:
loss_values.shape

(500,)